<a href="https://colab.research.google.com/github/RG2806/ai_spotle/blob/master/CNN-79.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/RG2806/ai_spotle.git

Cloning into 'ai_spotle'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 21 (delta 7), reused 2 (delta 0), pack-reused 0
Unpacking objects: 100% (21/21), done.


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from keras.models import Sequential #Initialise our neural network model as a sequential network
from keras.layers import Conv2D #Convolution operation
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation#Applies activation function
from keras.layers import Dropout#Prevents overfitting by randomly converting few outputs to zero
from keras.layers import MaxPooling2D # Maxpooling function
from keras.layers import Flatten # Converting 2D arrays into a 1D linear vector
from keras.layers import Dense # Regular fully connected neural network
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import accuracy_score

import pandas as pd
import keras 
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, GlobalAveragePooling2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.utils import np_utils
import keras.backend as k
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.regularizers import l2


In [3]:
def load_data(dataset_path):
  
  classes = ['Fear', 'Happy', 'Sad']  #We will be dealing with seven different types of emotions.

  data = []
  labels =[]

  with open(dataset_path, 'r') as file:
      for line_no, line in enumerate(file.readlines()):
          if 0 < line_no <= 35887:
            line = line.split(',')
            image_data = np.asarray([int(x) for x in line[1:]]).reshape(48, 48)#Creating a list out of the string then converting it into a 2-Dimensional numpy array.
            image_data =image_data.astype(np.uint8)/255.0
            data.append(image_data)
            labels.append(classes.index(line[0]))
            data.append(cv2.flip(image_data, 1))
            labels.append(classes.index(line[0]))
      data = np.expand_dims(data, -1)   
      #labels = to_categorical(labels, num_classes = 3)
    
      return np.array(data), np.array(labels)

In [4]:
dataset_path = "/content/ai_spotle/aithon2020_level2_traning.csv" 
train_data, train_labels = load_data(dataset_path)
#train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size = test_size,random_state = seed)

print("Number of images in Training set:", len(train_data))

Number of images in Training set: 21634


In [5]:
train_labels.shape

(21634,)

In [6]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=0.2, random_state=101)
y_train = np_utils.to_categorical(y_train, 3)
y_test = np_utils.to_categorical(y_test, 3)

In [7]:
num_features = 64
width, height = 48, 48
model = Sequential()

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1), data_format='channels_last', kernel_regularizer=l2(0.01)))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(2*2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*num_features, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 46, 46, 64)        36928     
_________________________________________________________________
batch_normalization (BatchNo (None, 46, 46, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 23, 23, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 23, 128)       73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 23, 23, 128)       5

In [8]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3, verbose=1)
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=8, verbose=1, mode='auto')
checkpointer = ModelCheckpoint('../input/model.h5', monitor='val_loss', verbose=1, save_best_only=True)

callbacks = [lr_reducer, early_stopper, checkpointer]

In [10]:
# With Augmentation
bs = 64
epochs = 100

aug = ImageDataGenerator(rotation_range=20, 
                         width_shift_range=0.1, 
                         height_shift_range=0.1, 
                         fill_mode="nearest")

H = model.fit(aug.flow(x_train, y_train, batch_size=bs),
                        validation_data=(x_test, y_test), 
                        steps_per_epoch=len(x_train)//bs,
                        callbacks=callbacks,
                        shuffle=True,
                        epochs=epochs)

Epoch 1/100
270/270 [==============================] - ETA: 0s - loss: 1.0344 - accuracy: 0.4753
Epoch 00001: val_loss did not improve from 1.06613
270/270 [==============================] - 27s 100ms/step - loss: 1.0344 - accuracy: 0.4753 - val_loss: 1.1875 - val_accuracy: 0.4756
Epoch 2/100
270/270 [==============================] - ETA: 0s - loss: 0.9816 - accuracy: 0.5196
Epoch 00002: val_loss improved from 1.06613 to 0.95980, saving model to ../input/model.h5
270/270 [==============================] - 27s 101ms/step - loss: 0.9816 - accuracy: 0.5196 - val_loss: 0.9598 - val_accuracy: 0.5313
Epoch 3/100
270/270 [==============================] - ETA: 0s - loss: 0.9201 - accuracy: 0.5543
Epoch 00003: val_loss did not improve from 0.95980
270/270 [==============================] - 27s 100ms/step - loss: 0.9201 - accuracy: 0.5543 - val_loss: 1.0153 - val_accuracy: 0.4786
Epoch 4/100
270/270 [==============================] - ETA: 0s - loss: 0.8624 - accuracy: 0.5842
Epoch 00004: val_l

In [11]:
scores = model.evaluate(x_test, y_test, batch_size=bs)
print("Loss: " + str(scores[0]))
print("Accuracy: " + str(scores[1]))

68/68 [==============================] - 2s 34ms/step - loss: 0.4858 - accuracy: 0.7908
Loss: 0.48579734563827515
Accuracy: 0.7908481359481812


In [12]:
model.save_weights("/content/model.h5")
print("Saved model to disk")

Saved model to disk
